<a href="https://colab.research.google.com/github/OlyKoek/MyStudy-AI-app/blob/colab/create_multimodal_embeding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# マルチモーダルなエンベディングモデルの作成

### Pytorchのパッケージと必要なライブラリをインストール
- torch   
- transformers from hugging face
- umap-learn : 可視化で使ってみたい。t-SNEよりも良いらしい
- matplotlib : ベクトル空間の可視化に使用

In [1]:
!pip install torch torchvision torchaudio --quiet
!pip install transformers umap-learn matplotlib pandas scikit-learn --quiet

やりたいことは、マルチモーダルな入力をベクトル空間に投影すること。
今回はできるだけ簡単に構築したいので、下記の構成。

- テキスト：TinyBERT系   
(最初all-MiniLM-L6-v2を使ったけど、日本語使えないことに気が追加ので一旦sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2に変更。東北大学が作った日本語専用のbert-base-japanese-v2もよさげだけど。一旦これで行く。)
- 画像：CLIP
- 音：MCFF

In [42]:
from transformers import AutoTokenizer, AutoModel
import torch

# read TinyBERT model
MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

# test sample text
texts = ["今日めっちゃ良かった楽しい","マジつまんね。つらい","今日は普通かな特に何もないし"]

# tokenize
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# view to tokenizerd text
print("-"*50)
text = texts[0]
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Tokenizeの出力確認\n\n 入力文章：{text} \nトークン文字：{tokens} \nトークンID: {ids}")
print("-"*50)

# input tokenized data to TinyBERT model
with torch.no_grad():
    outputs = model(**inputs)

# convert token to vector
embeddings = outputs.last_hidden_state.mean(dim=1)
print("\n文数 =",embeddings.shape[0]," ベクトル次元 =", embeddings.shape[1])

--------------------------------------------------
Tokenizeの出力確認

 入力文章：今日めっちゃ良かった楽しい 
トークン文字：['▁', '今日', 'め', 'っちゃ', '良かった', '楽しい'] 
トークンID: [6, 21834, 8421, 101963, 82989, 82672]
--------------------------------------------------

文数 = 3  ベクトル次元 = 384
